Lien : https://www.youtube.com/watch?v=VHqn7ufiilE

Dans ce fichier, on voit la comparaison pratiquement similaire d'un script pandas par rapport à un script polars

On voit par ailleurs l'intérêt de l'instruction lazy() pour le chargement des fichiers conséquents

Date : 07-03-23

In [50]:
import polars as pl
import pandas as pd

### Filtres avec Polars

In [63]:
# Récupération d'un fichier CSV
df = pl.read_csv('https://j.mp/iriscsv')

In [64]:
df.head()

sepal_length,sepal_width,petal_length,petal_width,species
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""setosa"""
4.9,3.0,1.4,0.2,"""setosa"""
4.7,3.2,1.3,0.2,"""setosa"""
4.6,3.1,1.5,0.2,"""setosa"""
5.0,3.6,1.4,0.2,"""setosa"""


In [65]:
# Filtre : données de la colonne 'petal_lenght' > à 5
df.filter(pl.col('petal_length') > 5).head()

sepal_length,sepal_width,petal_length,petal_width,species
f64,f64,f64,f64,str
6.0,2.7,5.1,1.6,"""versicolor"""
6.3,3.3,6.0,2.5,"""virginica"""
5.8,2.7,5.1,1.9,"""virginica"""
7.1,3.0,5.9,2.1,"""virginica"""
6.3,2.9,5.6,1.8,"""virginica"""


In [66]:

# Filtre : données de la colonne 'petal_lenght' > à 5
# regroupées par valeur de la colonne 'species'
# et classées par ordre croissant
# et afficher la somme dans toutes les autres colonnes
filtered = (df.filter(pl.col('petal_length') > 5)
 .groupby('species', maintain_order=True)
 .agg(pl.all().sum()))
filtered

species,sepal_length,sepal_width,petal_length,petal_width
str,f64,f64,f64,f64
"""versicolor""",6.0,2.7,5.1,1.6
"""virginica""",276.3,124.7,233.8,85.0


### Filtres avec Pandas

In [67]:
df = pd.read_csv('https://j.mp/iriscsv')

In [68]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [69]:
# Filtre : données de la colonne 'petal_lenght' > à 5
df.loc[df['petal_length'] > 5].head()

,sepal_length,sepal_width,petal_length,petal_width,species
83,6.0,2.7,5.1,1.6,versicolor
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica
102,7.1,3.0,5.9,2.1,virginica
103,6.3,2.9,5.6,1.8,virginica


In [70]:
# Filtre : données de la colonne 'petal_lenght' > à 5
# regroupées par valeur de la colonne 'species'
# et classées par ordre croissant
# et afficher la somme dans toutes les autres colonnes
filtered = (df.loc[df['petal_length'] > 5]
            .groupby('species').sum().reset_index().
            sort_values(by='species'))
filtered

6.16 ms ± 671 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Instruction lazy()

L'instruction lazy() permet de n'alimenter le fichier qu'après les traitements opérés sur le fichier, ce qui permet d'avoir un gain au niveau de la taille de la mémoire des données récupérées

In [72]:
filtered = (pl.read_csv('https://j.mp/iriscsv')
 .lazy() # execution du fichier chargés slt lors de l'exécut des req ci-après
 .filter(pl.col('petal_length') > 5)
 .groupby('species', maintain_order=True)
 .agg(pl.all().sum())
 .collect() # affichage des données traitées ci-avant
 )
filtered

620 ms ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
